<a href="https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/02_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
#| default_exp utils

# Utilities
> A bunch of helper functions

In [ ]:
#| export
import io, json, sys, re, csv, logging
import git
from types import SimpleNamespace
from logging import Formatter
from logging.handlers import RotatingFileHandler
from fastcore.foundation import L
from datetime import datetime
from rich import box
from rich.table import Table
from rich.console import Console
from rich.logging import RichHandler
from fastcore.basics import ifnone, listify, store_attr
from fastcore.xtras import run
from pathlib import Path
from execnb.nbio import read_nb

In [ ]:
this_nb = Path("02_utils.ipynb")
notebook = read_nb(this_nb)

## Logger
We will create a logger based on `rich.Table`, this way we get nice summaries at the end of execution

In [ ]:
#| export
LOGFORMAT = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"

In [ ]:
#| export
LOGFORMAT_RICH = "%(message)s"

In [ ]:
#| export
def create_table(columns=["Notebook Path", "Status", "Run Time", "Colab"], xtra_cols=None) -> Table:
    table = Table(show_header=True, header_style="bold magenta")
    table.box = box.SQUARE

    table.border_style = "bright_yellow"
    table.row_styles = ["none", "dim"]
    for col in columns + listify(xtra_cols):
        table.add_column(col)
    table.columns[1].style = "cyan"
    return table

In [ ]:
t = create_table(["a", "b"])
t

┌───┬───┐
│ a │ b │
├───┼───┤
└───┴───┘

In [ ]:
#| export
def remove_rich_format(text):
    "Remove rich fancy coloring"
    text = str(text)
    res = re.search(r"\](.*?)\[", text)
    if res is None:
        return text
    else:
        return res.group(1)

In [ ]:
s = "[green]Ok[/green]:heavy_check_mark:"
assert remove_rich_format(s) == "Ok"

In [ ]:
#| export
def _csv_to_md(csv_file_path, delimiter=";"):
    
    
    csv_dict = csv.DictReader(open(csv_file_path, encoding="UTF-8"), delimiter=delimiter)
    list_of_rows = [dict_row for dict_row in csv_dict]
    headers = list(list_of_rows[0].keys())
    md_string = " | "
    for header in headers:
        md_string += header + " |"

    md_string += "\n |"
    for i in range(len(headers)):
        md_string += "--- | "

    md_string += "\n"
    for row in list_of_rows:
        md_string += " | "
        for header in headers:
            md_string += row[header] + " | "
        md_string += "\n"
    return md_string

def csv_to_md(csv_file_path, delimiter=";"):
    "From csv file to markdown table, useful for github posting"
    md_string = _csv_to_md(csv_file_path, delimiter)
    output_file = Path(csv_file_path).with_suffix(".md")
    file = open(output_file, "w", encoding="UTF-8")
    file.write(md_string)
    file.close()

In [ ]:
print(_csv_to_md("test_data/file.csv", delimiter=","))

 | name |number |value |
 |--- | --- | --- | 
 | a | 1 | -100 | 
 | b | 2 | 0 | 
 | c | 3 | 2 | 



some fancy rich coloring

In [ ]:
#| export
STATUS = SimpleNamespace(
    ok="[green]Ok[/green]:heavy_check_mark:", fail="[red]Fail[/red]", skip="[green]Skipped[/green]:heavy_check_mark:"
)

In [ ]:
#| exporti
def _format_row(fname: Path, status: str, time: str, xtra_col=None, fname_only: bool = True) -> tuple:
    "Format one row for a rich.Table"

    formatted_status = getattr(STATUS, status.lower())
    fname = fname.name if fname_only else fname
    row = (str(fname), formatted_status, f"{int(time)}s")
    if len(listify(xtra_col)) > 0:
        row += (str(xtra_col),)
    return row

In [ ]:
_format_row(Path("/path/to/file"), "Ok", "1", fname_only=False)

('/path/to/file', '[green]Ok[/green]:heavy_check_mark:', '1s')

In [ ]:
#| export
class RichLogger:
    "A simple logger that logs to a file and the rich console"

    def __init__(self, columns, out_file="summary_table.csv", width=180):
        store_attr()
        self.data = []
        self.links = []
        self.console = Console(width=width, record=True)
        rh = RichHandler(console=self.console)
        rh.setFormatter(Formatter(LOGFORMAT_RICH))
        logging.basicConfig(
            level=logging.ERROR,
            format=LOGFORMAT,
            handlers=[
                rh,
                RotatingFileHandler("log.txt", maxBytes=1024 * 1024 * 10, backupCount=10),  # 10Mb
            ],
        )
        self.logger = logging.getLogger("rich")
        self.info(f"CONSOLE.is_terminal(): {self.console.is_terminal}")
        self.info(f"Writing output to {out_file}")

    def writerow(self, row, colab_link=None):
        self.data.append(row)
        self.links.append(colab_link)

    def writerow_incolor(self, fname, status, time, colab_link):
        "Same as write row, but color status"
        row = _format_row(fname, status, time)
        self.writerow(row, colab_link)

    def to_csv(self, out_file, delimiter=";", format_link=False):
        self.csv_file = open(out_file, "w", newline="")
        self.csv_writer = csv.writer(self.csv_file, delimiter=delimiter)
        # write header
        self.csv_writer.writerow(self.columns)
        for row, link in zip(self.data, self.links):
            if format_link:
                fname = self._format_colab_link_md(link, row[0])
            else:
                fname = row[0]
            self.csv_writer.writerow([fname] + [remove_rich_format(e) for e in row[1:]])
        self.csv_file.close()

    def to_table(self, enum=True):
        columns = (["#"] + self.columns) if enum else self.columns
        table = create_table(columns=columns)
        for i, (row, link) in enumerate(zip(self.data, self.links)):
            fname = self._format_colab_link(link, row[0])
            table.add_row(f"{i}", fname, *row[1:])
        self.console.print(table)

    def to_md(self, out_file):
        csv_file = Path(out_file).with_suffix(".csv")
        self.to_csv(csv_file)
        csv_to_md(csv_file)
        self.info(f"Output table saved to [red]{out_file}[/red]")

    @property
    def info(self):
        return self.logger.info

    @property
    def warning(self):
        return self.logger.warning

    @property
    def exception(self):
        return self.logger.exception

    @property
    def error(self):
        return self.logger.error

    @staticmethod
    def _format_colab_link(colab_link, fname):
        return f"[link={colab_link}]{fname}[link]"

    @staticmethod
    def _format_colab_link_md(colab_link, fname):
        return f"[{fname}]({colab_link})"

In [ ]:
l = RichLogger(["a", "b"])

In [ ]:
l.error("An Error!")

[09/26/22 20:29:24] ERROR    An Error!                                                                                                                               ]8;id=158508;file:///var/folders/sf/tgv7vcv96x557p38bvvp1ms40000gn/T/ipykernel_5571/1414801008.py\1414801008.py]8;;\:]8;id=823251;file:///var/folders/sf/tgv7vcv96x557p38bvvp1ms40000gn/T/ipykernel_5571/1414801008.py#1\1]8;;\

## Functions to make my life easier!
> A bunch of random functions to deal with notebooks, git, colab, wandb, etc...

In [ ]:
#| export
def is_nb(fname: Path):
    "filter files that are jupyter notebooks"
    return (fname.suffix == ".ipynb") and (not fname.name.startswith("_")) and (not "checkpoint" in str(fname))  and (not fname.is_symlink())

In [ ]:
assert is_nb(Path("02_utils.ipynb"))
assert not is_nb(Path("file.csv"))

In [ ]:
#| export
def find_nbs(path: Path):
    "Get all nbs on path recursively"
    path = Path(path).resolve()
    if is_nb(path):
        return [path]
    return L([nb.resolve() for nb in path.rglob("*.ipynb") if is_nb(nb)]).sorted()

In [ ]:
find_nbs("hola.ipynb")

[Path('/Users/tcapelle/wandb/nb_helpers/nbs/hola.ipynb')]

there should be 7 notebooks here

In [ ]:
assert len(find_nbs(Path("test_data"))) == 2

In [ ]:
#| export
def print_output(notebook):  # pragma: no cover
    "Print `notebook` in stdout for git things"
    output_stream = io.TextIOWrapper(sys.stdout.buffer, encoding="utf-8")
    x = json.dumps(notebook, sort_keys=True, indent=1, ensure_ascii=False)
    output_stream.write(x)
    output_stream.write("\n")
    output_stream.flush()

In [ ]:
# only works in terminal
# print_output(notebook)

In [ ]:
#| export
CellType = SimpleNamespace(code="code", md="markdown")

In [ ]:
#| export
def search_cell(cell, string) -> bool:
    "Search string in cell source, can be a list"
    source = listify(cell["source"])
    source = "".join(source)
    if string in source:
        return True
    return False

In [ ]:
cells = notebook["cells"]

In [ ]:
#check that we import rich
imports_cell = cells[3]
assert search_cell(imports_cell, "rich")

In [ ]:
#| export
def search_cells(nb, string: str = None, cell_type=CellType.code):
    "Get cells containing string, you can pass comma separated strings"
    strings = ifnone(string, "").replace(" ", "").split(",")
    cells = []
    for cell in nb["cells"]:
        if cell["cell_type"] == cell_type:
            if any([search_cell(cell, string) for string in strings]):
                cells.append(cell["source"])
    return cells

return the cells that contain the string in question

In [ ]:
search_cells(notebook, "search_cells")

['#| export\ndef search_cells(nb, string: str = None, cell_type=CellType.code):\n    "Get cells containing string, you can pass comma separated strings"\n    strings = ifnone(string, "").replace(" ", "").split(",")\n    cells = []\n    for cell in nb["cells"]:\n        if cell["cell_type"] == cell_type:\n            if any([search_cell(cell, string) for string in strings]):\n                cells.append(cell["source"])\n    return cells',
 'search_cells(notebook, "search_cells")',
 '#| export\ndef search_string_in_nb(nb, string: str = None, cell_type=CellType.code):\n    "Check if string is present in notebook cells, you can pass comma separated strings"\n    return len(search_cells(nb, string, cell_type)) > 0',
 'assert search_string_in_nb(notebook, "search_cells")',
 '#| export\ndef detect_imported_libs(notebook):\n    "Guess imported libs from notebook"\n    text_list = L(search_cells(notebook, "import,from")).concat()\n\n    # format lines\n    text_list = L([x.split("\\n") for x i

it is useful to filter out notebooks based on libraries or functions

In [ ]:
#| export
def search_string_in_nb(nb, string: str = None, cell_type=CellType.code):
    "Check if string is present in notebook cells, you can pass comma separated strings"
    return len(search_cells(nb, string, cell_type)) > 0

In [ ]:
assert search_string_in_nb(notebook, "search_cells")

Used imports and libraries

In [ ]:
#| export
def extract_libs(strings):
    "Automatically detect libraries imported in `strings`"

    after_import_regex = re.compile(r"^import\s([^\.]*)", re.VERBOSE)
    before_as_regex = re.compile(r"([^\s]*?)\sas\s", re.VERBOSE)
    between_from_import_regex = re.compile(r"^from\s(.*?)\simport", re.VERBOSE)

    def _search_with_regex(regex, string):
        res = regex.search(string)
        if res is not None:
            return res.group(1)
        else:
            return ""

    libs = []
    for string in strings:
        if "from" in string:
            string = _search_with_regex(between_from_import_regex, string).split(".")[0]
        else:
            string = _search_with_regex(after_import_regex, string)
            if "as" in string:
                string = _search_with_regex(before_as_regex, string)
        if string:
            libs.append(string.replace(" ", "").split(","))
    return L(libs).concat().unique()

In [ ]:
libs = ["import io, json, sys, re, csv, logging",
        "import git",
        "from types import SimpleNamespace",
        "from logging import Formatter"]

In [ ]:
libs

['import io, json, sys, re, csv, logging',
 'import git',
 'from types import SimpleNamespace',
 'from logging import Formatter']

In [ ]:
extract_libs(libs)

(#8) ['io','json','sys','re','csv','logging','git','types']

In [ ]:
#| export
def detect_imported_libs(notebook):
    "Guess imported libs from notebook"
    text_list = L(search_cells(notebook, "import,from")).concat()

    # format lines
    text_list = L([x.split("\n") for x in text_list]).concat()
    text_list = [line for line in text_list if (("from" in line) or ("import" in line))]

    return extract_libs(text_list)

In [ ]:
text_list = search_cells(notebook, "import,from")

In [ ]:
detect_imported_libs(notebook)

(#13) ['io','json','sys','re','csv','logging','git','types','fastcore','datetime'...]

## Git stuff
We deal with a bunch of guessing on the repo we are at, so we use pygit for this sorcery

In [ ]:
#| export
def get_repo(fname) -> git.Repo:
    try:
        repo = git.Repo(fname, search_parent_directories=True)
        return repo
    except Exception as e:
        raise Exception(f"Probably not in a git repo: {e}")

In [ ]:
repo = get_repo(this_nb)
assert type(repo) == git.Repo

In [ ]:
#| export
def git_current_branch(fname) -> str:
    "Get current git branch"
    repo = get_repo(fname)
    try:
        return repo.active_branch.name
    except Exception as e:
        return "master"

In [ ]:
git_current_branch(this_nb)

'main'

In [ ]:
#| export
def git_branches(repo: git.Repo, remote=True):
    "Get all remote or local banches"
    branches = set([b.name for b in repo.branches])
    remote_branches =  set([r.name.split("/")[-1] for r in repo.remote().refs])
    return branches.union(remote_branches) if remote else branches

In [ ]:
repo = get_repo(this_nb)
branches = git_branches(repo)
branches

{'HEAD',
 'action',
 'clean-up',
 'docker',
 'fix-ci',
 'fixes',
 'git',
 'github',
 'improve_test_nb',
 'main',
 'muellerzr/main',
 'nbdev',
 'traceback'}

In [ ]:
#| export
def git_main_name(fname) -> str:
    "Get the name of master/main branch"
    repo = get_repo(fname)
    branches = git_branches(repo)
    return "main" if "main" in branches else "master"

In [ ]:
assert git_main_name(this_nb) == 'main', f"It's not called main? {branches}"

In [ ]:
#| export
def _get_github_repo_remote(repo_url):
    if "git@" in repo_url:
        github_repo = re.search(r".com:(.*).git", repo_url).group(1)
    else:
        github_repo = re.search(r".com/(.*)", repo_url).group(1)
    return github_repo

In [ ]:
https_url = 'https://github.com/org/repo'
assert _get_github_repo_remote(https_url) == "org/repo"

git_url = 'git@github.com:wandb/nb_helpers.git'
assert _get_github_repo_remote(git_url) == "wandb/nb_helpers"

In [ ]:
#| export
def git_origin_repo(fname):
    "Get github repo name from `fname`"
    repo = get_repo(fname)
    repo_url = repo.remote().url

    # check if ssh or html
    if repo_url != "":
        return _get_github_repo_remote(repo_url)
    else:
        raise Exception(f"Not in a valid github repo: {fname=}")

In [ ]:
repo_name = git_origin_repo(this_nb)
assert repo_name.split("/")[1] == 'nb_helpers', f"The repo name is {repo_name}"

In [ ]:
#| export
def git_local_repo(fname):
    "Get local github repo path"
    repo = get_repo(fname)
    return Path(repo.git_dir).parent.resolve()

In [ ]:
git_local_repo(this_nb)

Path('/Users/tcapelle/wandb/nb_helpers')

In [ ]:
#| export
def git_last_commit(fname):
    "Gets the last commit on fname"
    repo = get_repo(fname)
    return repo.commit().hexsha

In [ ]:
git_last_commit(this_nb)

'4f20b6d5cc627cf710c4c88ac5bf6577d2f444b7'

## Other random stuff

In [ ]:
#| export
def today():
    "datetime object containing current date and time"
    now = datetime.now()

    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    return dt_string

get todays date in nice format

In [ ]:
today()

'26/09/2022 20:25:59'